In [ ]:
# INSTALL DEPENDENCIES
!pip install -q transformers datasets

# IMPORTS
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # ✅ Corrected import
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# DEVICE CONFIGURATION
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# LOAD DATA (update your path if needed)
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("electronics_reviews.csv")

# ENSURE CORRECT COLUMNS — change these if your column names differ
text_col = "review_text"
label_col = "label"

print("Columns:", df.columns.tolist())

text_col = 'review_text'
label_col = 'sentiment'  # not 'label'

print("Sample:", df[[text_col, label_col]].head())

# LABEL ENCODING (if not already numeric)
if df[label_col].dtype == 'object':
    df[label_col] = df[label_col].astype('category').cat.codes

# TRAIN-VAL-TEST SPLIT
from sklearn.model_selection import train_test_split
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df[text_col].tolist(), df[label_col].tolist(), test_size=0.2, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42)

# TOKENIZER
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# CUSTOM DATASET
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# DATASETS
train_dataset = ReviewDataset(train_texts, train_labels, tokenizer)
val_dataset = ReviewDataset(val_texts, val_labels, tokenizer)
test_dataset = ReviewDataset(test_texts, test_labels, tokenizer)

# DATALOADERS
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# MODEL
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(set(df[label_col])))
model.to(device)

# OPTIMIZER
optimizer = AdamW(model.parameters(), lr=2e-5)

# TRAINING LOOP
epochs = 3
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    model.train()
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

# EVALUATION FUNCTION
def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            labels = batch["labels"].cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)
    acc = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds)
    return acc, report

# VALIDATION RESULTS
val_acc, val_report = evaluate(model, val_loader)
print("\nValidation Accuracy:", val_acc)
print("\nValidation Report:\n", val_report)

# TEST RESULTS
test_acc, test_report = evaluate(model, test_loader)
print("\nTest Accuracy:", test_acc)
print("\nTest Report:\n", test_report)
